In [0]:
print("hello world")

hello world


In [0]:
from pyspark import SparkConf,SparkContext 
conf=SparkConf().setAppName("Read Files")
context=SparkContext.getOrCreate(conf=conf)

In [0]:
text=context.textFile('/FileStore/tables/sample.txt')

In [0]:
text #doesnt print anything, because it'll not do the action unless triggered, reffering to lazy execution

Out[4]: /FileStore/tables/sample.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [0]:
text.collect()

Out[5]: ['1 2 3 4 5', '3 4 5 66 77', '12 43 7 8', '12 12 33']

Transformations
1. Map
2. FlatMap
3. filter
4. distinct
5. group by key
6. reduce by key
7. repartition
8. coalesce

In [0]:
rdd=text.map(lambda x: x.split(' '))
#creates a new rdd which splits the output based on spaces, note that map function always creates new rdd
rdd.collect()

Out[6]: [['1', '2', '3', '4', '5'],
 ['3', '4', '5', '66', '77'],
 ['12', '43', '7', '8'],
 ['12', '12', '33']]

In [0]:
#alternative for lambda function
def new(x):
    return x.split(' ')

rdd1=text.map(new)
rdd1.collect()

Out[7]: [['1', '2', '3', '4', '5'],
 ['3', '4', '5', '66', '77'],
 ['12', '43', '7', '8'],
 ['12', '12', '33']]

In [0]:
def newint(x):
    #x -> '1 2 3 4 5'
    l=x.split() 
    #l -> ['1','2','3','4','5']
    l2=[]
    for i in l:
        l2.append(int(i)+2) #converting the numbers to integer from string and adding them with 2
    return l2

rdd2=text.map(newint)
rdd2.collect()
    
    

Out[8]: [[3, 4, 5, 6, 7], [5, 6, 7, 68, 79], [14, 45, 9, 10], [14, 14, 35]]

In [0]:
flatmap=text.flatMap(lambda x:x.split())
flatmap.collect()

Out[11]: ['1',
 '2',
 '3',
 '4',
 '5',
 '3',
 '4',
 '5',
 '66',
 '77',
 '12',
 '43',
 '7',
 '8',
 '12',
 '12',
 '33']

In [0]:
rddfilter=text.filter(lambda x:x!='12 12 33')
rddfilter.collect()

Out[13]: ['1 2 3 4 5', '3 4 5 66 77', '12 43 7 8']

In [0]:
def fil(x):
    if x=='12 12 33':
        return False
    else:
        return True
rddfil=text.filter(fil)
rddfil.collect()

Out[14]: ['1 2 3 4 5', '3 4 5 66 77', '12 43 7 8']

In [0]:
distinct=flatmap.distinct() #we are choosing flatmap so that it parses through each and every element
distinct.collect()

Out[16]: ['1', '4', '66', '77', '12', '8', '33', '2', '3', '5', '43', '7']

In [0]:
flatmap.distinct().collect()

Out[17]: ['1', '4', '66', '77', '12', '8', '33', '2', '3', '5', '43', '7']

In [0]:
keyvalue=flatmap.map(lambda x:(x,len(x)))
keyvalue.collect()  #normal key value pair is taken here

Out[22]: [('1', 1),
 ('2', 1),
 ('3', 1),
 ('4', 1),
 ('5', 1),
 ('3', 1),
 ('4', 1),
 ('5', 1),
 ('66', 2),
 ('77', 2),
 ('12', 2),
 ('43', 2),
 ('7', 1),
 ('8', 1),
 ('12', 2),
 ('12', 2),
 ('33', 2)]

In [0]:
keyvalue.groupByKey().collect() #here the key value pairs are grouped together

Out[24]: [('1', <pyspark.resultiterable.ResultIterable at 0x7fda636b6730>),
 ('4', <pyspark.resultiterable.ResultIterable at 0x7fda68076d60>),
 ('66', <pyspark.resultiterable.ResultIterable at 0x7fda68076bb0>),
 ('77', <pyspark.resultiterable.ResultIterable at 0x7fda63a6f5b0>),
 ('12', <pyspark.resultiterable.ResultIterable at 0x7fda63a6f970>),
 ('8', <pyspark.resultiterable.ResultIterable at 0x7fda63a6f9a0>),
 ('33', <pyspark.resultiterable.ResultIterable at 0x7fda63a6fcd0>),
 ('2', <pyspark.resultiterable.ResultIterable at 0x7fda63a6fc70>),
 ('3', <pyspark.resultiterable.ResultIterable at 0x7fda63a6f340>),
 ('5', <pyspark.resultiterable.ResultIterable at 0x7fda63a6ff10>),
 ('43', <pyspark.resultiterable.ResultIterable at 0x7fda63a6f250>),
 ('7', <pyspark.resultiterable.ResultIterable at 0x7fda63a6f820>)]

In [0]:
keyvalue.groupByKey().mapValues(list).collect() #the values for the keys are showed in a list

Out[25]: [('1', [1]),
 ('4', [1, 1]),
 ('66', [2]),
 ('77', [2]),
 ('12', [2, 2, 2]),
 ('8', [1]),
 ('33', [2]),
 ('2', [1]),
 ('3', [1, 1]),
 ('5', [1, 1]),
 ('43', [2]),
 ('7', [1])]

In [0]:
keyvalue.reduceByKey(lambda x,y:x+y).collect()

Out[26]: [('1', 1),
 ('4', 2),
 ('66', 2),
 ('77', 2),
 ('12', 6),
 ('8', 1),
 ('33', 2),
 ('2', 1),
 ('3', 2),
 ('5', 2),
 ('43', 2),
 ('7', 1)]

In [0]:
text.getNumPartitions()

Out[31]: 2

In [0]:
text1=text.repartition(5)
text1.getNumPartitions()
#this increasing or decreasing of partitions is done to increase the speed of the code, could help in parallelization

Out[35]: 5

In [0]:
text2=text1.flatMap(lambda x:x.split(' ')).map(lambda x:(x,1))
text2.saveAsTextFile('/FileStore/tables/output/5partitions')

In [0]:
text1.getNumPartitions()

Out[41]: 5

In [0]:
text2=text2.coalesce(3)
text2.saveAsTextFile('/FileStore/tables/output/3partitions')

Actions
1. collect
2. count
3. count by value

In [0]:
text.count()

Out[27]: 4

In [0]:
flatmap.count()

Out[28]: 17

In [0]:
flatmap.countByValue()

Out[29]: defaultdict(int,
            {'1': 1,
             '2': 1,
             '3': 2,
             '4': 2,
             '5': 2,
             '66': 1,
             '77': 1,
             '12': 3,
             '43': 1,
             '7': 1,
             '8': 1,
             '33': 1})